In [1]:
from transformers import pipeline
from nltk.metrics.distance import edit_distance, jaccard_distance
from nltk.util import ngrams

In [2]:
unmasker = pipeline('fill-mask', model='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
with open("words_alpha.txt","r") as f:
    dictionary = [word.strip() for word in f.readlines()]

In [4]:
def find_mask(sentence):
    #sentence = 'I have been thinking of having lonchh with you'
    input_text = ""
    new_sentence = ""
    c = 0
    pos = -1
    no_mask = 0
    words = sentence.split()
    for i in range(len(words)):
        c = 0

        if words[i].lower() in dictionary:
            c = 1
            input_text += words[i]
            input_text += " "
            continue
        elif c == 0:
            pos = i
            input_text += '[MASK] '
    c = 0
    if '[MASK]' in input_text:
        if input_text.count('[MASK]',0,len(input_text))>1:
            pos = 0
            new_words = input_text.split()
            for i in range(len(new_words)):
                if new_words[i] == '[MASK]':
                    c +=1
                    if c == 1:
                        new_sentence += new_words[i]
                        new_sentence +=" "
                        pos = i
                        continue
                    else:
                        new_sentence +=""
                else:

                    new_sentence += new_words[i]
                    new_sentence +=" "
            #print(new_sentence,pos)
            no_mask = 1
            predictor(sentence,new_sentence,pos,no_mask)
        else:
            no_mask = 0
            predictor(sentence,input_text,pos,no_mask)
    

In [5]:
def predictor(sentence,new_sentence,pos,no_mask):
    words = sentence.split()
    predictions = unmasker(new_sentence)
    min_dist = 100
    #print(pos)
    final_pred = ''
    for i in predictions:

        pred_word = i['token_str']
        
        distance = edit_distance(pred_word, words[pos]) 
        #print(pred_word, distance)        
        if distance < min_dist:
            min_dist = distance
            final_pred = pred_word

    print("\nIncorrect word: ",words[pos], " Corrected word: ",final_pred)
    words[pos] = final_pred

    
    predicted_sentence = " ".join([str(word) for word in words])
    find_mask(predicted_sentence)
    if no_mask == 0:
        print("\nPredicted Sentence:")
        print(predicted_sentence)
    predicted_sentence = ""

In [8]:
test_sentences = ['I have been thinking of having lonchh with you',
                "who am I taalking with tell me your nmae",
                  "find out who he is",
                 "do you feeeel better now I haerd you were not eaping well",
                 "do u wanna coll up your parents"] 
for text in test_sentences:
    print("\n\n\nInput Sentence: ")
    print(text)
    find_mask(text)




Input Sentence: 
I have been thinking of having lonchh with you

Incorrect word:  lonchh  Corrected word:  lunch

Predicted Sentence:
I have been thinking of having lunch with you



Input Sentence: 
who am I taalking with tell me your nmae

Incorrect word:  taalking  Corrected word:  talking

Incorrect word:  nmae  Corrected word:  name

Predicted Sentence:
who am I talking with tell me your name



Input Sentence: 
find out who he is



Input Sentence: 
do you feeeel better now I haerd you were not eaping well

Incorrect word:  feeeel  Corrected word:  feel

Incorrect word:  haerd  Corrected word:  said

Incorrect word:  eaping  Corrected word:  eating

Predicted Sentence:
do you feel better now I said you were not eating well



Input Sentence: 
do u wanna coll up your parents
